In [6]:
%%time
%load_ext autoreload

from analysis_utils import *
from joblib import Parallel, delayed
import multiprocessing  
import numpy as np
import pandas as pd

#Growth rate = 0.6
#Computes the translation latencies from our model with simulations containing
#1 to 14 cognate ternary complexes in the voxel (14 to 42 not included because
#the probability of voxels with that number of cognates is ~0)
path = "./data/"
ensmbl_latency_dict = dict()
rxndiff=dict()
scalingFactor = 200
NR_scaledrates = {'k1r':718*200/scalingFactor,'k2f':1475*200/scalingFactor,'k2r_nr':1120*200/scalingFactor,'k3_nr':6*200/scalingFactor,'k4':209*200/scalingFactor}


data = "191003_2140/"
for i in range(1,7):
    sim_i = computeTransportRxnTimes(path+data,100*(i-1),100*(i-1)+100,cogtRNANum=i, ribosomeNum = 7,scaling=scalingFactor,NR_scaling = NR_scaledrates)
    ensmbl_latency_dict[i] = (CellLatencies(sim_i))

data = "191105_1738/"
for i in range(7,16):
    sim_i = computeTransportRxnTimes(path+data, 100*(i-1-6),100*(i-1-6)+100,cogtRNANum=i, ribosomeNum = 7,scaling=scalingFactor,NR_scaling = NR_scaledrates)
    ensmbl_latency_dict[i] = (CellLatencies(sim_i))
    
#Experimentally obtained codon usage frequencies and tRNA abundances at growth rate = 2.5 dbl/hr


gr06_Codon = np.array([4.81, 2.71, 38.29, 35.62, 16.57, 53.1, 24.25, 28.72, 21.4, 15.87, 31.31, 11.25, 30.33, 22.13, 28.85, 19.8, 0.09, 1.12, 3.99, 11.97, 12.08, 44.43, 9.79, 27.95, 22.37, 0.93, 21.38, 36.68, 7.53, 3.48, 13.88, 26.51, 9.76, 0.31, 4.23, 5.29, 10.68, 16.2, 6.63, 6.13, 12.55, 22.68, 6.05, 3.89, 13.12, 11.15, 1.75, 1.32, 31.12, 22.25, 29.24, 10.19, 9.23, 13.9, 60.13, 2.15, 5.7, 6.19, 29.51, 6.52, 4.99, 3.32])
gr06_pCodon = gr06_Codon/np.sum(gr06_Codon)

n_cores = 16

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
CPU times: user 43.6 s, sys: 752 ms, total: 44.4 s
Wall time: 45.2 s


In [7]:
#GFP gene optimization

tRNA_distributions = {}
tRNA_tags = ['Ala1B', 'Ala2', 'Arg2', 'Arg3', 'Arg4', 'Arg5', 'Asn', 'Asp1', 'Cys', 'Gln1', 'Gln2', 'Glu2', 'Gly2', 'Gly3', 'His', 'Ile1', 'Leu1', 'Leu2', 'Leu3', 'Leu4', 'Leu5', 'Lys', 'Met m', 'Phe', 'Pro1', 'Pro2', 'Pro3', 'Sec', 'Ser1', 'Ser2', 'Ser3', 'Ser5', 'Thr1', 'Thr2', 'Thr3', 'Thr4', 'Trp', 'Tyr1+Tyr2', 'Val1', 'Val2A+2B']
codon_tags = ['GGG', 'GGA', 'GGU', 'GGC', 'GAG', 'GAA', 'GAU', 'GAC', 'GUG', 'GUA', 'GUU', 'GUC', 'GCG', 'GCA', 'GCU', 'GCC', 'AGG', 'AGA', 'AGU', 'AGC', 'AAG', 'AAA', 'AAU', 'AAC', 'AUG', 'AUA', 'AUU', 'AUC', 'ACG', 'ACA', 'ACU', 'ACC', 'UGG', 'UGA', 'UGU', 'UGC', 'UAU', 'UAC', 'UUG', 'UUA', 'UUU', 'UUC', 'UCG', 'UCA', 'UCU', 'UCC', 'CGG', 'CGA', 'CGU', 'CGC', 'CAG', 'CAA', 'CAU', 'CAC', 'CUG', 'CUA', 'CUU', 'CUC', 'CCG', 'CCA', 'CCU', 'CCC']


from analysis_utils import *
gene_map, pCodon_GFP = singlegene_to_genemap('./data/tables/sfGFP_RED20.csv')
#wildtype
ptRNA_red20 = pd.read_excel('./data/tables/tRNAValues_RED20.xlsx',header=None)
ptRNA_red20 = ptRNA_red20[6]/sum(ptRNA_red20[6])

codon_elongt = compute_codon_elongt(ptRNA_red20, len(pCodon_GFP), ensmbl_latency_dict)
avg_elongt_wt = compute_gene_elongt_genemap(gene_map['sfGFP_RED20'],codon_elongt)
print('elongt wildtype', avg_elongt_wt)
tRNA_distributions['GFP_wt'] = [avg_elongt_wt,dict(zip(tRNA_tags,ptRNA_red20)),dict(zip(codon_tags,pCodon_GFP))]

#Uniform
ptRNA_red20_uniform = pd.read_excel('./data/tables/tRNAValues_RED20.xlsx',header=None)
ptRNA_red20_uniform = ptRNA_red20_uniform[6]/sum(ptRNA_red20_uniform[6])
ptRNA_red20_uniform[ptRNA_red20_uniform>0] = 1/20

codon_elongt = compute_codon_elongt(ptRNA_red20_uniform, len(pCodon_GFP), ensmbl_latency_dict)
avg_elongt_uniform = compute_gene_elongt_genemap(gene_map['sfGFP_RED20'],codon_elongt)
print('elongt uniform', avg_elongt_uniform)
tRNA_distributions['GFP_uniform'] = [avg_elongt_uniform,dict(zip(tRNA_tags,ptRNA_red20_uniform)),dict(zip(codon_tags,pCodon_GFP))]


#GFP Codon-weighted correlated
ptRNA_red20 = pd.read_excel('./data/tables/tRNAValues_RED20.xlsx',header=None)
ptRNA_red20 = ptRNA_red20[6]/sum(ptRNA_red20[6])
ptRNA_dict, pcodon_dict, codon_dict, codonLabels, pCodon, tRNA_dict = cognateDistrib(ptRNA_red20,pCodon_GFP,extra2=True)
tRNA_sum_dict = tRNA_dict.copy()
tRNA_weightedbycodon = list()
for keys in tRNA_dict.keys():
    tRNA_sum_dict[keys] = (sum([pcodon_dict[tRNA_key] for tRNA_key in tRNA_dict[keys]]))
tRNA_weightedbycodon = list(tRNA_sum_dict.values())/sum(tRNA_sum_dict.values())
tRNA_sorted_indices = np.flip(tRNA_weightedbycodon.argsort())
tRNA_sorted = np.array(tRNA_tags)[tRNA_sorted_indices]

##Assign monotonic increase to GFP codon-sorted tRNA
ptRNA_sorted = tRNA_weightedbycodon[tRNA_sorted_indices]
sortedtRNAdict = dict(zip(tRNA_sorted, ptRNA_sorted))
resortedtRNA = [sortedtRNAdict[tRNA] for tRNA in tRNA_tags]

codon_elongt = compute_codon_elongt(resortedtRNA, len(pCodon_GFP), ensmbl_latency_dict)
avg_elongt_codonweightedcorrelated = compute_gene_elongt_genemap(gene_map['sfGFP_RED20'],codon_elongt)
print('elongt codon-weighted correlated', avg_elongt_codonweightedcorrelated)

tRNA_distributions['GFP_codonweightedcorrelated'] = [avg_elongt_codonweightedcorrelated,dict(zip(tRNA_tags,resortedtRNA)),dict(zip(codon_tags,pCodon_GFP))]


#GFP Codon-weighted anticorrelated
tRNA_tags = ['Ala1B', 'Ala2', 'Arg2', 'Arg3', 'Arg4', 'Arg5', 'Asn', 'Asp1', 'Cys', 'Gln1', 'Gln2', 'Glu2', 'Gly2', 'Gly3', 'His', 'Ile1', 'Leu1', 'Leu2', 'Leu3', 'Leu4', 'Leu5', 'Lys', 'Met m', 'Phe', 'Pro1', 'Pro2', 'Pro3', 'Sec', 'Ser1', 'Ser2', 'Ser3', 'Ser5', 'Thr1', 'Thr2', 'Thr3', 'Thr4', 'Trp', 'Tyr1+Tyr2', 'Val1', 'Val2A+2B']
codon_tags = ['GGG', 'GGA', 'GGU', 'GGC', 'GAG', 'GAA', 'GAU', 'GAC', 'GUG', 'GUA', 'GUU', 'GUC', 'GCG', 'GCA', 'GCU', 'GCC', 'AGG', 'AGA', 'AGU', 'AGC', 'AAG', 'AAA', 'AAU', 'AAC', 'AUG', 'AUA', 'AUU', 'AUC', 'ACG', 'ACA', 'ACU', 'ACC', 'UGG', 'UGA', 'UGU', 'UGC', 'UAU', 'UAC', 'UUG', 'UUA', 'UUU', 'UUC', 'UCG', 'UCA', 'UCU', 'UCC', 'CGG', 'CGA', 'CGU', 'CGC', 'CAG', 'CAA', 'CAU', 'CAC', 'CUG', 'CUA', 'CUU', 'CUC', 'CCG', 'CCA', 'CCU', 'CCC']

ptRNA_red20 = pd.read_excel('./data/tables/tRNAValues_RED20.xlsx',header=None)
ptRNA_red20 = ptRNA_red20[6]/sum(ptRNA_red20[6])
ptRNA_dict, pcodon_dict, codon_dict, codonLabels, pCodon, tRNA_dict = cognateDistrib(ptRNA_red20,pCodon_GFP,extra2=True)
tRNA_sum_dict = tRNA_dict.copy()
tRNA_weightedbycodon = list()
for keys in tRNA_dict.keys():
    tRNA_sum_dict[keys] = (sum([pcodon_dict[tRNA_key] for tRNA_key in tRNA_dict[keys]]))
tRNA_weightedbycodon = list(tRNA_sum_dict.values())/sum(tRNA_sum_dict.values())
tRNA_sorted_indices = np.flip(tRNA_weightedbycodon.argsort())
tRNA_sorted = np.array(tRNA_tags)[tRNA_sorted_indices]

##Assign monotonic increase to GFP codon-sorted tRNA
ptRNA_sorted = tRNA_weightedbycodon[tRNA_sorted_indices]
ptRNA_sorted_flipped = np.append(np.flip([ptRNA for ptRNA in ptRNA_sorted if ptRNA != 0]),np.array([ptRNA for ptRNA in ptRNA_sorted if ptRNA == 0]))
sortedtRNAdict = dict(zip(tRNA_sorted, ptRNA_sorted_flipped))
resortedtRNA = [sortedtRNAdict[tRNA] for tRNA in tRNA_tags]

codon_elongt = compute_codon_elongt(resortedtRNA, len(pCodon_GFP), ensmbl_latency_dict)
avg_elongt_codonweightedanticorrelated = compute_gene_elongt_genemap(gene_map['sfGFP_RED20'],codon_elongt)
print('elongt codon-weighted anticorrelated', avg_elongt_codonweightedanticorrelated)

tRNA_distributions['GFP_codonweightedanticorrelated'] = [avg_elongt_codonweightedanticorrelated,dict(zip(tRNA_tags,resortedtRNA)),dict(zip(codon_tags,pCodon_GFP))]


########### Genetic algorithm fast and slow #############

elongt_list_fast =np.load('./data/UniformRandomtRNA_minimalcode-sfGFP_2ksample_fasttRNA_elongt.npy')
ptRNA_list_fast=np.load('./data/UniformRandomtRNA_minimalcode-sfGFP_2ksample_fasttRNA_ptRNA.npy')
elongt_fastest = np.min(elongt_list_fast)
ptRNA_min = ptRNA_list_fast[np.argmin(elongt_list_fast)]
tRNA_distributions['GFP_gafast'] = [elongt_fastest,dict(zip(tRNA_tags,ptRNA_min)),dict(zip(codon_tags,pCodon_GFP))]
print('GFP fast', tRNA_distributions['GFP_gafast'])

elongt_list_slow =np.load('./data/UniformRandomtRNA_minimalcode-sfGFP_2ksample_slowtRNA_elongt.npy')
ptRNA_list_slow =np.load('./data/UniformRandomtRNA_minimalcode-sfGFP_2ksample_slowtRNA_ptRNA.npy')
elongt_slowest = np.max(elongt_list_slow)
ptRNA_max = ptRNA_list_slow[np.argmax(elongt_list_slow)]
tRNA_distributions['GFP_gaslow'] = [elongt_slowest,dict(zip(tRNA_tags,ptRNA_max)),dict(zip(codon_tags,pCodon_GFP))]
print('GFP slow', tRNA_distributions['GFP_gaslow'])



######### Export all to pickle ##############

import pickle
with open('tRNA_distributions-GFP.pickle', 'wb') as handle:
    pickle.dump(tRNA_distributions, handle, protocol=pickle.HIGHEST_PROTOCOL)

/Users/akshay/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


elongt wildtype 445.3649754139615


/Users/akshay/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


elongt uniform 447.51563710314616


/Users/akshay/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


elongt codon-weighted correlated 418.23850443362596


/Users/akshay/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


elongt codon-weighted anticorrelated 552.0634361422375
GFP fast [387.7651971202895, {'Ala1B': 0.022793312494091493, 'Ala2': 0.0, 'Arg2': 0.015704347420851535, 'Arg3': 0.0, 'Arg4': 0.0, 'Arg5': 0.0, 'Asn': 0.0688885821035588, 'Asp1': 0.0764291155458389, 'Cys': 0.012213740457866095, 'Gln1': 0.0, 'Gln2': 0.047175681543304836, 'Glu2': 0.07313049037133579, 'Gly2': 0.10196738620914622, 'Gly3': 0.0, 'His': 0.03931078582791767, 'Ile1': 0.04148028159565396, 'Leu1': 0.0, 'Leu2': 0.0, 'Leu3': 0.07487850515799128, 'Leu4': 0.0, 'Leu5': 0.0, 'Lys': 0.10256557259283926, 'Met m': 0.012213740457866095, 'Phe': 0.045358775647735564, 'Pro1': 0.012213740457866095, 'Pro2': 0.0, 'Pro3': 0.0, 'Sec': 0.0, 'Ser1': 0.06367052522321266, 'Ser2': 0.0, 'Ser3': 0.0, 'Ser5': 0.0, 'Thr1': 0.08238070239063185, 'Thr2': 0.0, 'Thr3': 0.0, 'Thr4': 0.0, 'Trp': 0.012213740457866095, 'Tyr1+Tyr2': 0.03474307776603868, 'Val1': 0.0, 'Val2A+2B': 0.060667896278387166}, {'GGG': 0.0, 'GGA': 0.09205020920502092, 'GGU': 0.0, 'GGC': 0.0

In [10]:
tRNA_distributions

{'GFP_wt': [445.3649754139615,
  {'Ala1B': 0.07175572518996332,
   'Ala2': 0.0,
   'Arg2': 0.09770992366292877,
   'Arg3': 0.0,
   'Arg4': 0.0,
   'Arg5': 0.0,
   'Asn': 0.021374045801265666,
   'Asp1': 0.04427480915976458,
   'Cys': 0.021374045801265666,
   'Gln1': 0.0,
   'Gln2': 0.030534351144665236,
   'Glu2': 0.08549618320506266,
   'Gly2': 0.10534351144909505,
   'Gly3': 0.0,
   'His': 0.012213740457866095,
   'Ile1': 0.0717557251899633,
   'Leu1': 0.0,
   'Leu2': 0.0,
   'Leu3': 0.12824427480759398,
   'Leu4': 0.0,
   'Leu5': 0.0,
   'Lys': 0.030534351144665236,
   'Met m': 0.013740458015099355,
   'Phe': 0.015267175572332618,
   'Pro1': 0.025954198472965453,
   'Pro2': 0.0,
   'Pro3': 0.0,
   'Sec': 0.0,
   'Ser1': 0.054961832060397425,
   'Ser2': 0.0,
   'Ser3': 0.0,
   'Ser5': 0.0,
   'Thr1': 0.047328244274231115,
   'Thr2': 0.0,
   'Thr3': 0.0,
   'Thr4': 0.0,
   'Trp': 0.015267175572332618,
   'Tyr1+Tyr2': 0.027480916298901,
   'Val1': 0.0,
   'Val2A+2B': 0.0793893127196410

In [9]:
a = list(tRNA_distributions['GFP_codonweightedanticorrelated'][1].values())
np.min(np.array(a)[np.nonzero(a)])

0.003745318352059925

In [8]:
a = np.array(ptRNA_red20)
np.min(np.array(a)[np.nonzero(a)])

0.012213740457866095

np.min(ptRNA_red20[np.nonzero(ptRNA_red20)]

In [18]:
#mRFP gene optimization

tRNA_distributions = {}
tRNA_tags = ['Ala1B', 'Ala2', 'Arg2', 'Arg3', 'Arg4', 'Arg5', 'Asn', 'Asp1', 'Cys', 'Gln1', 'Gln2', 'Glu2', 'Gly2', 'Gly3', 'His', 'Ile1', 'Leu1', 'Leu2', 'Leu3', 'Leu4', 'Leu5', 'Lys', 'Met m', 'Phe', 'Pro1', 'Pro2', 'Pro3', 'Sec', 'Ser1', 'Ser2', 'Ser3', 'Ser5', 'Thr1', 'Thr2', 'Thr3', 'Thr4', 'Trp', 'Tyr1+Tyr2', 'Val1', 'Val2A+2B']
codon_tags = ['GGG', 'GGA', 'GGU', 'GGC', 'GAG', 'GAA', 'GAU', 'GAC', 'GUG', 'GUA', 'GUU', 'GUC', 'GCG', 'GCA', 'GCU', 'GCC', 'AGG', 'AGA', 'AGU', 'AGC', 'AAG', 'AAA', 'AAU', 'AAC', 'AUG', 'AUA', 'AUU', 'AUC', 'ACG', 'ACA', 'ACU', 'ACC', 'UGG', 'UGA', 'UGU', 'UGC', 'UAU', 'UAC', 'UUG', 'UUA', 'UUU', 'UUC', 'UCG', 'UCA', 'UCU', 'UCC', 'CGG', 'CGA', 'CGU', 'CGC', 'CAG', 'CAA', 'CAU', 'CAC', 'CUG', 'CUA', 'CUU', 'CUC', 'CCG', 'CCA', 'CCU', 'CCC']


from analysis_utils import *
gene_map, pCodon_mRFP = singlegene_to_genemap('./data/tables/mRFP_RED20.csv')
#wildtype
ptRNA_red20 = pd.read_excel('./data/tables/tRNAValues_RED20.xlsx',header=None)
ptRNA_red20 = ptRNA_red20[6]/sum(ptRNA_red20[6])

codon_elongt = compute_codon_elongt(ptRNA_red20, len(pCodon_GFP), ensmbl_latency_dict)
avg_elongt_wt = compute_gene_elongt_genemap(gene_map['mRFP_RED20'],codon_elongt)
print('elongt wildtype', avg_elongt_wt)
tRNA_distributions['mRFP_RED20'] = [avg_elongt_wt,dict(zip(tRNA_tags,ptRNA_red20)),dict(zip(codon_tags,pCodon_mRFP))]

#Uniform
ptRNA_red20_uniform = pd.read_excel('./data/tables/tRNAValues_RED20.xlsx',header=None)
ptRNA_red20_uniform = ptRNA_red20_uniform[6]/sum(ptRNA_red20_uniform[6])
ptRNA_red20_uniform[ptRNA_red20_uniform>0] = 1/20

codon_elongt = compute_codon_elongt(ptRNA_red20_uniform, len(pCodon_GFP), ensmbl_latency_dict)
avg_elongt_uniform = compute_gene_elongt_genemap(gene_map['mRFP_RED20'],codon_elongt)
print('elongt uniform', avg_elongt_uniform)
tRNA_distributions['mRFP_uniform'] = [avg_elongt_uniform,dict(zip(tRNA_tags,ptRNA_red20_uniform)),dict(zip(codon_tags,pCodon_mRFP))]


#GFP Codon-weighted correlated

ptRNA_red20 = pd.read_excel('./data/tables/tRNAValues_RED20.xlsx',header=None)
ptRNA_red20 = ptRNA_red20[6]/sum(ptRNA_red20[6])
ptRNA_dict, pcodon_dict, codon_dict, codonLabels, pCodon, tRNA_dict = cognateDistrib(ptRNA_red20,pCodon_mRFP,extra2=True)
tRNA_sum_dict = tRNA_dict.copy()
tRNA_weightedbycodon = list()
for keys in tRNA_dict.keys():
    tRNA_sum_dict[keys] = (sum([pcodon_dict[tRNA_key] for tRNA_key in tRNA_dict[keys]]))
tRNA_weightedbycodon = list(tRNA_sum_dict.values())/sum(tRNA_sum_dict.values())
tRNA_sorted_indices = np.flip(tRNA_weightedbycodon.argsort())
tRNA_sorted = np.array(tRNA_tags)[tRNA_sorted_indices]

##Assign monotonic increase to GFP codon-sorted tRNA
ptRNA_sorted = tRNA_weightedbycodon[tRNA_sorted_indices]
sortedtRNAdict = dict(zip(tRNA_sorted, ptRNA_sorted))
resortedtRNA = [sortedtRNAdict[tRNA] for tRNA in tRNA_tags]

codon_elongt = compute_codon_elongt(resortedtRNA, len(pCodon_GFP), ensmbl_latency_dict)
avg_elongt_codonweightedcorrelated = compute_gene_elongt_genemap(gene_map['mRFP_RED20'],codon_elongt)
print('elongt codon-weighted correlated', avg_elongt_codonweightedcorrelated)

tRNA_distributions['mRFP_codonweightedcorrelated'] = [avg_elongt_codonweightedcorrelated,dict(zip(tRNA_tags,resortedtRNA)),dict(zip(codon_tags,pCodon_mRFP))]


#GFP Codon-weighted anticorrelated
tRNA_tags = ['Ala1B', 'Ala2', 'Arg2', 'Arg3', 'Arg4', 'Arg5', 'Asn', 'Asp1', 'Cys', 'Gln1', 'Gln2', 'Glu2', 'Gly2', 'Gly3', 'His', 'Ile1', 'Leu1', 'Leu2', 'Leu3', 'Leu4', 'Leu5', 'Lys', 'Met m', 'Phe', 'Pro1', 'Pro2', 'Pro3', 'Sec', 'Ser1', 'Ser2', 'Ser3', 'Ser5', 'Thr1', 'Thr2', 'Thr3', 'Thr4', 'Trp', 'Tyr1+Tyr2', 'Val1', 'Val2A+2B']
codon_tags = ['GGG', 'GGA', 'GGU', 'GGC', 'GAG', 'GAA', 'GAU', 'GAC', 'GUG', 'GUA', 'GUU', 'GUC', 'GCG', 'GCA', 'GCU', 'GCC', 'AGG', 'AGA', 'AGU', 'AGC', 'AAG', 'AAA', 'AAU', 'AAC', 'AUG', 'AUA', 'AUU', 'AUC', 'ACG', 'ACA', 'ACU', 'ACC', 'UGG', 'UGA', 'UGU', 'UGC', 'UAU', 'UAC', 'UUG', 'UUA', 'UUU', 'UUC', 'UCG', 'UCA', 'UCU', 'UCC', 'CGG', 'CGA', 'CGU', 'CGC', 'CAG', 'CAA', 'CAU', 'CAC', 'CUG', 'CUA', 'CUU', 'CUC', 'CCG', 'CCA', 'CCU', 'CCC']

ptRNA_red20 = pd.read_excel('./data/tables/tRNAValues_RED20.xlsx',header=None)
ptRNA_red20 = ptRNA_red20[6]/sum(ptRNA_red20[6])
ptRNA_dict, pcodon_dict, codon_dict, codonLabels, pCodon, tRNA_dict = cognateDistrib(ptRNA_red20,pCodon_mRFP,extra2=True)
tRNA_sum_dict = tRNA_dict.copy()
tRNA_weightedbycodon = list()
for keys in tRNA_dict.keys():
    tRNA_sum_dict[keys] = (sum([pcodon_dict[tRNA_key] for tRNA_key in tRNA_dict[keys]]))
tRNA_weightedbycodon = list(tRNA_sum_dict.values())/sum(tRNA_sum_dict.values())
tRNA_sorted_indices = np.flip(tRNA_weightedbycodon.argsort())
tRNA_sorted = np.array(tRNA_tags)[tRNA_sorted_indices]

##Assign monotonic increase to GFP codon-sorted tRNA
ptRNA_sorted = tRNA_weightedbycodon[tRNA_sorted_indices]
ptRNA_sorted_flipped = np.append(np.flip([ptRNA for ptRNA in ptRNA_sorted if ptRNA != 0]),np.array([ptRNA for ptRNA in ptRNA_sorted if ptRNA == 0]))
sortedtRNAdict = dict(zip(tRNA_sorted, ptRNA_sorted_flipped))
resortedtRNA = [sortedtRNAdict[tRNA] for tRNA in tRNA_tags]

codon_elongt = compute_codon_elongt(resortedtRNA, len(pCodon_GFP), ensmbl_latency_dict)
avg_elongt_codonweightedanticorrelated = compute_gene_elongt_genemap(gene_map['mRFP_RED20'],codon_elongt)
print('elongt codon-weighted anticorrelated', avg_elongt_codonweightedanticorrelated)

tRNA_distributions['mRFP_codonweightedanticorrelated'] = [avg_elongt_codonweightedanticorrelated,dict(zip(tRNA_tags,resortedtRNA)),dict(zip(codon_tags,pCodon_mRFP))]


########### Genetic algorithm fast and slow #############

elongt_list_fast =np.load('./data/UniformRandomtRNA_minimalcode-mRFP_2ksample_fasttRNA_elongt.npy')
ptRNA_list_fast=np.load('./data/UniformRandomtRNA_minimalcode-mRFP_2ksample_fasttRNA_ptRNA.npy')
elongt_fastest = np.min(elongt_list_fast)
ptRNA_min = ptRNA_list_fast[np.argmin(elongt_list_fast)]
tRNA_distributions['mRFP_gafast'] = [elongt_fastest,dict(zip(tRNA_tags,ptRNA_min)),dict(zip(codon_tags,pCodon_mRFP))]

elongt_list_slow =np.load('./data/UniformRandomtRNA_minimalcode-mRFP_2ksample_slowtRNA_elongt.npy')
ptRNA_list_slow =np.load('./data/UniformRandomtRNA_minimalcode-mRFP_2ksample_slowtRNA_ptRNA.npy')
elongt_slowest = np.max(elongt_list_slow)
ptRNA_max = ptRNA_list_slow[np.argmax(elongt_list_slow)]
tRNA_distributions['mRFP_gaslow'] = [elongt_slowest,dict(zip(tRNA_tags,ptRNA_max)),dict(zip(codon_tags,pCodon_mRFP))]


######### Export all to pickle ##############

import pickle
with open('tRNA_distributions-mRFP.pickle', 'wb') as handle:
    pickle.dump(tRNA_distributions, handle, protocol=pickle.HIGHEST_PROTOCOL)

elongt wildtype 173.44537806058804
elongt uniform 179.08324128594163
elongt codon-weighted correlated 162.42524525183458
elongt codon-weighted anticorrelated 196.00742981509876


In [ ]:
#Max elongt check w/ all tRNA zero but one tRNA = 1 probability, as control
ptRNA_red20_zeros = pd.read_excel('./data/tables/tRNAValues_RED20.xlsx',header=None)
ptRNA_red20_zeros = ptRNA_red20_zeros[6]/sum(ptRNA_red20_zeros[6])
ptRNA_red20_zeros[ptRNA_red20_zeros>0] = 0
ptRNA_red20_zeros[1] = 1

codon_elongt = compute_codon_elongt(ptRNA_red20_zeros, len(pCodon_GFP), ensmbl_latency_dict)
avg_elongt_max = compute_gene_elongt_genemap(gene_map['sfGFP_RED20'],codon_elongt)
print('elongt max', avg_elongt_max)

In [2]:
import pickle
with open(r"tRNA_distributions-GFP.pickle", "rb") as input_file:
    e = pickle.load(input_file)

In [34]:
e['GFP_gafast']

NameError: name 'e' is not defined

In [10]:
#GFP GA slow fast optimization

tRNA_distributions = {}
from analysis_utils import *
gene_map, pCodon_GFP = singlegene_to_genemap('./data/tables/sfGFP_RED20.csv')

tRNA_tags = ['Ala1B', 'Ala2', 'Arg2', 'Arg3', 'Arg4', 'Arg5', 'Asn', 'Asp1', 'Cys', 'Gln1', 'Gln2', 'Glu2', 'Gly2', 'Gly3', 'His', 'Ile1', 'Leu1', 'Leu2', 'Leu3', 'Leu4', 'Leu5', 'Lys', 'Met m', 'Phe', 'Pro1', 'Pro2', 'Pro3', 'Sec', 'Ser1', 'Ser2', 'Ser3', 'Ser5', 'Thr1', 'Thr2', 'Thr3', 'Thr4', 'Trp', 'Tyr1+Tyr2', 'Val1', 'Val2A+2B']
codon_tags = ['GGG', 'GGA', 'GGU', 'GGC', 'GAG', 'GAA', 'GAU', 'GAC', 'GUG', 'GUA', 'GUU', 'GUC', 'GCG', 'GCA', 'GCU', 'GCC', 'AGG', 'AGA', 'AGU', 'AGC', 'AAG', 'AAA', 'AAU', 'AAC', 'AUG', 'AUA', 'AUU', 'AUC', 'ACG', 'ACA', 'ACU', 'ACC', 'UGG', 'UGA', 'UGU', 'UGC', 'UAU', 'UAC', 'UUG', 'UUA', 'UUU', 'UUC', 'UCG', 'UCA', 'UCU', 'UCC', 'CGG', 'CGA', 'CGU', 'CGC', 'CAG', 'CAA', 'CAU', 'CAC', 'CUG', 'CUA', 'CUU', 'CUC', 'CCG', 'CCA', 'CCU', 'CCC']

import numpy as np
elongt_list_fast =np.load('./data/UniformRandomtRNA_minimalcode-sfGFP_2ksample_fasttRNA_elongt.npy')
ptRNA_list_fast=np.load('./data/UniformRandomtRNA_minimalcode-sfGFP_2ksample_fasttRNA_ptRNA.npy')
elongt_fastest = np.min(elongt_list_fast)
ptRNA_min = ptRNA_list_fast[np.argmin(elongt_list_fast)]
tRNA_distributions['GFP_gafast'] = [elongt_fastest,dict(zip(tRNA_tags,ptRNA_min)),dict(zip(codon_tags,pCodon_GFP))]

elongt_list_slow =np.load('./data/UniformRandomtRNA_minimalcode-sfGFP_2ksample_slowtRNA_elongt.npy')
ptRNA_list_slow =np.load('./data/UniformRandomtRNA_minimalcode-sfGFP_2ksample_slowtRNA_ptRNA.npy')
elongt_slowest = np.max(elongt_list_slow)
ptRNA_max = ptRNA_list_slow[np.argmax(elongt_list_slow)]
tRNA_distributions['GFP_gaslow'] = [elongt_slowest,dict(zip(tRNA_tags,ptRNA_max)),dict(zip(codon_tags,pCodon_GFP))]




In [13]:
ptRNA_list_slow[2]

array([0.00606658, 0.        , 0.00228968, 0.        , 0.        ,
       0.        , 0.00326699, 0.0043185 , 0.01338964, 0.        ,
       0.00289458, 0.00679118, 0.00462071, 0.        , 0.00381738,
       0.00304735, 0.        , 0.        , 0.00259271, 0.        ,
       0.        , 0.00221045, 0.01085204, 0.01972292, 0.62466653,
       0.        , 0.        , 0.        , 0.0024463 , 0.        ,
       0.        , 0.        , 0.00356713, 0.        , 0.        ,
       0.        , 0.27650342, 0.00404843, 0.        , 0.00288748])

In [54]:
gene_map, pCodon_GFP = singlegene_to_genemap('./data/tables/sfGFP_RED20.csv')
ptRNA = [0.0059797, 0.,         0.07493208, 0.,        0.     ,    0.,
 0.07962803, 0.07506485, 0.00259992 ,0.        , 0.0160313  ,0.07524019,
 0.08669846 ,0.         ,0.04863804 ,0.05502781 ,0.         ,0.,
 0.08228277 ,0.         ,0.         ,0.07989156 ,0.00939848 ,0.04610216,
 0.00408228 ,0.         ,0.         ,0.         ,0.06124321 ,0.,
 0.         ,0.         ,0.09661121 ,0.         ,0.         ,0.,
 0.0139354  ,0.01994671 ,0.         ,0.06666584]

computeElongationLatency(ptRNA,pCodon_GFP,ensmbl_latency_dict)

([159.10309724668667], [7.614853533501633])

In [57]:
gene_map, pCodon_GFP = singlegene_to_genemap('./data/tables/sfGFP_RED20.csv')
ptRNA = np.ones(40)*0.00001

computeElongationLatency(ptRNA,pCodon_GFP,ensmbl_latency_dict)

([216.99230779540477], [15.364193395706891])